In [ ]:
years = np.arange(1990, 2002)
TZ = -6

ds_temp = ds.convert_calendar("noleap")
ds_temp["time"] =  ds_temp.indexes["time"].shift(TZ, "h")

ds_m = []
for i in range(years.shape[0]):
    ds_m.append( ds.where( ( (ds["time"].dt.month==(i+1)) &
        (ds["time"].dt.year==years[i]) ), drop = True ) )
ds_tmy = xr.concat(ds_m, dim = "time")

In [ ]:
df_tmy = ds_tmy.to_dataframe()
time = df_tmy.index.get_level_values("time")
m = 1
front = df_tmy.loc[ ( ( ( time.month.isin([m]) & time.hour.isin(range(18, 24)) )
    | ( time.month.isin([1 + 1]) & time.hour.isin(range(0, 6)) ) ), lat, lon ), v ]